In [1]:
import sys
sys.path.append('../scripts/')
from mcl import *

In [2]:
class ResetMcl(Mcl):   ###resetmcl2
    def __init__(self, envmap, init_pose, num, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}, \
                 distance_dev_rate=0.14, direction_dev=0.05, alpha_threshold=0.001): #alpha_thresholdを追加
        super().__init__(envmap, init_pose, num, motion_noise_stds, distance_dev_rate, direction_dev)
        self.alpha_threshold = alpha_threshold  #閾値(self.alphasから書き換える）
        
    def random_reset(self): #追加
         for p in self.particles:
                p.pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
                p.weight = 1/len(self.particles)
        
    def resetting(self): #追加
        self.random_reset()
        
    def observation_update(self, observation):
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
            
        self.set_ml()   #以下書き換え

        if sum([p.weight for p in self.particles]) < self.alpha_threshold: #αの計算
            self.resetting()
        else:
            self.resampling() 

In [3]:
if __name__ == "aaa":#'__main__':  ###resetmcl2kidnap
    time_interval = 0.1
    world = World(30, time_interval) 

    ### 地図を生成して3つランドマークを追加 ###
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)          

    ### ロボットを作る ###
    initial_pose = np.array([0, 0, 0]).T
    pf = ResetMcl(m, initial_pose, 100)
    circling = MclAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
#  r = Robot(initial_pose, sensor=Camera(m), agent=circling, color="red")   #誘拐なし
    r = Robot(np.array([0,0,0]).T, sensor=Camera(m), agent=circling, expected_kidnap_time=10.0, color="red") #誘拐あり
    world.append(r)

    world.draw()
#    world.ani.save('/tmp/anm.gif', writer='imagemagick', fps=1)

In [4]:
def trial(animation): ###mclkidnap1test
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation) 

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)

    ## ロボットを作る ##
    init_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
    robot_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
    pf = ResetMcl(m, init_pose, 100)
    a = MclAgent(time_interval, 0.2, 10.0/180*math.pi, pf)
    r = Robot(robot_pose, sensor=Camera(m), agent=a, color="red")
    world.append(r)

    world.draw()
    
    return (r.pose, pf.ml.pose)

In [6]:
if __name__ == '__main__': ###mclglobal1exec
    ok = 0
    for i in range(1000):
        actual, estm = trial(False)
        diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
        print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
        if diff <= 1.0:
            ok += 1

    ok

0 真値: [ 1.51632284 -4.20851149  2.78585723] 推定値 [ 2.73432472 -3.92768365  3.89343972] 誤差: 1.2499571409263384
1 真値: [3.3080905  0.39257137 8.94006139] 推定値 [3.5443743  0.44672327 2.85473611] 誤差: 0.24240970568139356
2 真値: [3.66737622 4.16121621 7.73867617] 推定値 [2.72203754 4.25461475 2.06378451] 誤差: 0.9499413137977952
3 真値: [1.66405421 0.52725526 5.28062214] 推定値 [ 1.45619215  0.50645127 -0.91484589] 誤差: 0.2089005612018927
4 真値: [ 5.01971561 -0.4846782   3.01996718] 推定値 [-0.04964024 -1.79473613 -2.84515004] 誤差: 5.235897305691168
5 真値: [ 0.03668331 -4.55530966  1.94191658] 推定値 [-0.64198901 -2.59507155  1.06220035] 誤差: 2.0743985963600267
6 真値: [-2.40189265  3.29825553  5.19068882] 推定値 [-2.39496669 -0.411973    3.38251395] 誤差: 3.710234999942304
7 真値: [-2.76482183 -2.41758525  2.24683696] 推定値 [-2.47941893 -3.53203929  2.29185642] 誤差: 1.1504184542971885
8 真値: [-5.30553461 -0.96528898  4.76045913] 推定値 [-1.61822466  0.67557217  6.45847946] 誤差: 4.035923689245261
9 真値: [ 3.14357496 -1.47123148  7.81

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


19 真値: [-3.70232105 -1.98085564  7.87443114] 推定値 [-3.27488455 -2.44551068  1.66807157] 誤差: 0.6313527279557423
20 真値: [-1.29094174  1.3942941   7.38333155] 推定値 [-1.30341461  0.20887599  1.37895737] 誤差: 1.185483721490525
21 真値: [-6.05993478  0.46305435  4.3838709 ] 推定値 [-5.54539174 -0.47166447  5.51167725] 誤差: 1.0669835106256969
22 真値: [-2.7062233  -1.09717811  6.68846133] 推定値 [-2.71643039 -1.11561659  6.75033573] 誤差: 0.021075158393732755
23 真値: [ 3.02985569 -3.169454    0.68436027] 推定値 [ 2.66017717 -3.63682887  0.60246115] 誤差: 0.5959039165940375
24 真値: [4.22691386 2.53467301 3.55003708] 推定値 [ 0.41447182 -4.52619474 -1.28676115] 誤差: 8.024373347079777
25 真値: [2.19479792 4.22488625 4.04601706] 推定値 [3.50203286 3.0177453  4.0056861 ] 誤差: 1.7793404541222593
26 真値: [-0.9327086  -3.23460048  6.44994369] 推定値 [-1.78513265 -4.29896403  6.69996542] 誤差: 1.3636335767907084
27 真値: [-0.59216718 -2.84333248  5.31100756] 推定値 [ 0.07950954 -1.32628002  4.66031666] 誤差: 1.6590954728898435
28 真値: [-3.99297813

95 真値: [-0.10424792 -5.19599391  1.67775577] 推定値 [-0.49419505 -1.45467071  0.29081681] 誤差: 3.761589823649528
96 真値: [ 1.83622237 -4.41553681  7.16991716] 推定値 [ 1.07242161 -1.83993724  4.79377535] 誤差: 2.686466968438098
97 真値: [-3.0293646  -4.29990279  8.68392674] 推定値 [-0.27748174  1.298099    1.16271795] 誤差: 6.237826811592676
98 真値: [ 0.84601289 -1.0317714   6.21584306] 推定値 [-0.10463254 -1.5029583   0.26001858] 誤差: 1.061010761754398
99 真値: [-0.00922313  4.6465687   1.99048949] 推定値 [-0.30268656  0.80985487  2.81359207] 誤差: 3.847920712403818
100 真値: [-3.09920684  4.0293069   5.00361897] 推定値 [-2.07625457  1.95048319  3.79460514] 誤差: 2.316881382383344
101 真値: [-0.77403945 -2.30407043  3.68704455] 推定値 [-1.36006411 -1.91467069  3.39494071] 誤差: 0.7036029182427505
102 真値: [-1.60698343 -1.86352608  4.56696305] 推定値 [-1.90130072 -1.94965273  4.03825356] 誤差: 0.30666018283069846
103 真値: [-3.04305061 -0.0631433   7.07898918] 推定値 [-3.10585612  0.32395566  6.98454659] 誤差: 0.39216085031746767
104 真値: [-

170 真値: [ 0.67292167 -5.0559895   5.91639689] 推定値 [-6.06558308  3.2825554   4.89155351] 誤差: 10.720950398880374
171 真値: [ 1.466431   -0.5815171   2.99311067] 推定値 [0.83066552 0.24469169 2.56431981] 誤差: 1.0425059679327693
172 真値: [1.32828131 1.13618465 3.23910238] 推定値 [1.71320293 0.78963009 3.19060026] 誤差: 0.5179427710147819
173 真値: [-4.72966222  3.26944694  4.88567785] 推定値 [-4.48531981  3.59376507  4.67612233] 誤差: 0.4060609140075607
174 真値: [0.83601642 5.91648341 4.10664151] 推定値 [-1.00828932  2.76579142  4.41602866] 誤差: 3.6507976796655845
175 真値: [-2.0516076   0.87435622  8.13145799] 推定値 [-0.8913035   0.9244811   1.95945767] 誤差: 1.161386291268135
176 真値: [ 2.98249579 -5.80496013  7.30252516] 推定値 [ 4.32093342 -5.31690126  7.72124084] 誤差: 1.424646174726488
177 真値: [-2.75176346 -1.25423776  7.92288257] 推定値 [-2.3597408  -0.7962015   8.04490573] 誤差: 0.602892177101433
178 真値: [-1.94955817  5.20187448  4.59063542] 推定値 [-1.68092553  5.1688733   4.51091266] 誤差: 0.2706521302620128
179 真値: [-4.7248

...

697 真値: [0.08908218 1.48533388 7.65547092] 推定値 [-0.37924378  1.38253016  1.29701945] 誤差: 0.47947660387956964
698 真値: [ 3.57754604 -3.454632    6.78633283] 推定値 [ 2.89778541 -0.93869769  1.85632594] 誤差: 2.606146580094429
699 真値: [ 1.63244775 -0.11478936  5.49366651] 推定値 [ 1.81759638  0.04387786 -0.91346979] 誤差: 0.24383458197552843
700 真値: [1.34802758 1.1015866  8.7304123 ] 推定値 [1.64038325 2.99577659 2.74627897] 誤差: 1.916618787027411
701 真値: [-2.19489332  3.31333842  5.02662816] 推定値 [-2.60874064  3.6358345   5.24529954] 誤差: 0.5246649658916784
702 真値: [-1.26744057 -4.25342379  5.8014821 ] 推定値 [-1.59329848 -4.90846519  5.98614079] 誤差: 0.7316164344070621
703 真値: [-0.46018368  3.26153906  8.25384007] 推定値 [1.00521676 0.33961405 2.71222156] 誤差: 3.268798591207484
704 真値: [-2.48309726 -2.22023474  7.66489399] 推定値 [-2.56005056 -1.51408725  7.7317323 ] 誤差: 0.7103281542287804
705 真値: [ 2.71912909 -1.73528065  2.61503721] 推定値 [-1.59000318  1.061926    7.75242177] 誤差: 5.137410434504074
706 真値: [-3.312

772 真値: [ 1.37755387 -1.53724855  4.38919337] 推定値 [ 1.44432459 -1.55570098  4.33567872] 誤差: 0.06927352884735008
773 真値: [3.94525318 1.55156454 3.73437039] 推定値 [-2.67004548 -1.04198912 -0.99808422] 誤差: 7.105539883153855
774 真値: [-2.41154893  5.04279037  3.81500116] 推定値 [-3.68795741 -1.06733909  1.29996606] 誤差: 6.242026967222538
775 真値: [0.6789906  4.94012972 3.25386128] 推定値 [1.76931796 3.4834126  2.88711088] 誤差: 1.819570967321678
776 真値: [-0.23965884 -3.00303231  8.60550459] 推定値 [-0.43208354 -3.42426169  8.84655594] 誤差: 0.46309984048927727
777 真値: [ 3.52119377 -0.74814553  4.28875389] 推定値 [-0.57220611 -2.92032009  0.15615853] 誤差: 4.634033325243324
778 真値: [-0.46097095 -2.14279571  3.90097815] 推定値 [ 1.04692359  2.33627204 -1.70428871] 誤差: 4.7260759470714
779 真値: [ 1.83029593 -3.16473543  8.46591834] 推定値 [ 2.27075698 -2.82895731  4.85162385] 誤差: 0.553852768587294
780 真値: [-2.1852896  -3.78426133  5.94669604] 推定値 [-2.48078984 -3.46521919  5.94217711] 誤差: 0.4348658181670608
781 真値: [0.13818

848 真値: [-3.92090401 -0.44139394  1.73240492] 推定値 [-2.1473318   0.49449007  2.5865244 ] 誤差: 2.005352149157762
849 真値: [0.07724184 2.86201769 6.06067207] 推定値 [0.07817223 3.23341547 5.91740047] 誤差: 0.37139893955267916
850 真値: [-4.87805058 -3.60416012  6.14883756] 推定値 [-3.65664304 -4.66100908  5.79999157] 誤差: 1.6151675170153672
851 真値: [4.48397248 1.43788048 7.57291092] 推定値 [0.38161628 1.12732119 2.54806864] 誤差: 4.114094492267452
852 真値: [-0.37820529 -0.48601056  2.78383855] 推定値 [-0.56504645 -0.53472453  2.91283504] 誤差: 0.19308721418564048
853 真値: [-2.28543599  1.98005339  4.04556982] 推定値 [-2.50962412  1.86540344  3.9628593 ] 誤差: 0.2518033603025696
854 真値: [-1.14575957 -0.90864632  6.87597298] 推定値 [-2.00812568 -1.12803324  0.4760596 ] 誤差: 0.8898347756867339
855 真値: [1.42849905 0.93500076 3.86783434] 推定値 [ 1.13149173 -1.16964338 -2.84786549] 誤差: 2.1254976660812805
856 真値: [-5.71592596  1.95115729  5.67327709] 推定値 [-4.51877266  0.62467181  6.88087921] 誤差: 1.7868239253459037
857 真値: [ 3.8767

923 真値: [-1.39683639 -3.00413633  5.77659945] 推定値 [-1.48079676 -3.27523358  5.83844992] 誤差: 0.28380109727791075
924 真値: [-2.1600653   1.56888109  1.75234005] 推定値 [ 0.91222677 -1.37742412  2.27588779] 誤差: 4.256723262294067
925 真値: [3.09260291 4.27640379 4.74840098] 推定値 [1.73722635 2.42442425 0.51406351] 誤差: 2.294967071439236
926 真値: [3.86379025 0.79654596 8.33407809] 推定値 [1.59548636 1.02182024 0.96050794] 誤差: 2.279462888104629
927 真値: [2.84962757 1.0001799  2.24296609] 推定値 [2.79053473 1.03922996 2.23746185] 誤差: 0.07082987311629121
928 真値: [-1.3677418  -0.04185271  2.65715501] 推定値 [-1.71290366 -0.51695539  2.30575224] 誤差: 0.5872472002377261
929 真値: [-3.01966718 -3.27171401  2.89389263] 推定値 [ 2.02692777  1.60314443 -2.12045607] 誤差: 7.01657790288369
930 真値: [ 2.99676697 -4.92924562  5.28904857] 推定値 [-0.13858978 -1.8978136   2.81171002] 誤差: 4.361197323824724
931 真値: [2.36318025 3.92295389 8.05244709] 推定値 [2.35952933 3.74080125 2.00663605] 誤差: 0.18218922882062832
932 真値: [ 1.59876369 -1.8856

998 真値: [2.19690309 0.25566225 7.94596893] 推定値 [1.80652814 0.46958728 1.57413899] 誤差: 0.445147745492093
999 真値: [-4.95177426  2.26070225  7.31808117] 推定値 [-3.74335377  0.40285514  2.94692116] 誤差: 2.2162752418947185


In [7]:
ok

446